In [5]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/dwd-icon"
params = {
	"latitude": [54.6791, 52.2915, 53.0451, 50.7913, 51.1278, 50.4283, 51.162, 53.633, 47.801, 47.6952, 52.209, 53.7123, 49.503, 52.381, 54.1803, 49.2128, 54.3279, 54.528, 48.8281, 49.7479, 48.4024, 49.7704, 47.421, 48.5451],
	"longitude": [13.4344, 10.4464, 8.7981, 12.872, 13.7543, 12.9536, 14.951, 9.988, 11.011, 9.1307, 14.118, 7.1519, 11.0549, 13.062, 12.0808, 7.1077, 8.6031, 9.549, 9.2, 6.6583, 11.6946, 9.9576, 10.9848, 13.3532],
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation", "cloud_cover", "wind_speed_10m"],
	"timezone": ["Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin", "Europe/Berlin"],
	"past_days": 31
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()/60} m")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(5).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(6).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["precipitation"] = hourly_precipitation
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe

Coordinates 54.68000030517578°N 13.439998626708984°E
Elevation 40.0 m asl
Timezone b'Europe/Berlin' b'CEST'
Timezone difference to GMT+0 120.0 m


,date,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,cloud_cover,wind_speed_10m
0,2024-05-25 22:00:00+00:00,13.490000,92.0,12.215392,12.185705,0.0,100.0,13.684735
1,2024-05-25 23:00:00+00:00,13.290000,92.0,12.017374,12.424212,0.0,81.0,10.239453
2,2024-05-26 00:00:00+00:00,13.440000,92.0,12.165888,12.467648,0.0,97.0,11.298495
3,2024-05-26 01:00:00+00:00,13.240000,92.0,11.967870,12.106417,0.0,80.0,11.966953
4,2024-05-26 02:00:00+00:00,12.889999,96.0,12.267310,12.187695,0.0,95.0,9.673221
...,...,...,...,...,...,...,...,...
907,2024-07-02 17:00:00+00:00,18.389999,61.0,10.734322,17.515385,0.0,27.0,7.568566
908,2024-07-02 18:00:00+00:00,18.090000,63.0,10.936496,17.525196,0.0,40.0,5.860375
909,2024-07-02 19:00:00+00:00,17.440001,69.0,11.692225,17.187851,0.0,31.0,5.315336
910,2024-07-02 20:00:00+00:00,16.539999,78.0,12.689614,16.532207,0.0,21.0,5.860375


In [4]:
hourly_dataframe

,date,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,wind_speed_10m
0,2024-05-25 23:00:00+00:00,16.595499,82.0,13.509601,16.902561,0.0,5.623380
1,2024-05-26 00:00:00+00:00,15.595500,91.0,14.131086,16.396255,0.0,3.758510
2,2024-05-26 01:00:00+00:00,14.795500,97.0,14.323655,15.524284,0.0,4.735060
3,2024-05-26 02:00:00+00:00,14.095500,95.0,13.306198,14.558578,0.0,4.072935
4,2024-05-26 03:00:00+00:00,13.695499,93.0,12.583660,14.175747,0.0,2.276840
...,...,...,...,...,...,...,...
907,2024-07-02 18:00:00+00:00,19.069500,51.0,8.697532,17.274546,0.0,9.957108
908,2024-07-02 19:00:00+00:00,18.069500,56.0,9.156044,16.675785,0.0,8.049845
909,2024-07-02 20:00:00+00:00,16.869501,63.0,9.781803,15.876669,0.0,6.489992
910,2024-07-02 21:00:00+00:00,15.769500,69.0,10.094409,15.069830,0.0,5.091168
